In [ ]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from keras.models import load_model
import os
import shutil

In [ ]:
from google.colab import files
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!kaggle datasets download -d navoneel/brain-mri-images-for-brain-tumor-detection

Saving kaggle.json to kaggle (1).json
brain-mri-images-for-brain-tumor-detection.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip brain-mri-images-for-brain-tumor-detection.zip

Archive:  brain-mri-images-for-brain-tumor-detection.zip
  inflating: brain_tumor_dataset/no/1 no.jpeg  
  inflating: brain_tumor_dataset/no/10 no.jpg  
  inflating: brain_tumor_dataset/no/11 no.jpg  
  inflating: brain_tumor_dataset/no/12 no.jpg  
  inflating: brain_tumor_dataset/no/13 no.jpg  
  inflating: brain_tumor_dataset/no/14 no.jpg  
  inflating: brain_tumor_dataset/no/15 no.jpg  
  inflating: brain_tumor_dataset/no/17 no.jpg  
  inflating: brain_tumor_dataset/no/18 no.jpg  
  inflating: brain_tumor_dataset/no/19 no.jpg  
  inflating: brain_tumor_dataset/no/2 no.jpeg  
  inflating: brain_tumor_dataset/no/20 no.jpg  
  inflating: brain_tumor_dataset/no/21 no.jpg  
  inflating: brain_tumor_dataset/no/22 no.jpg  
  inflating: brain_tumor_dataset/no/23 no.jpg  
  inflating: brain_tumor_dataset/no/24 no.jpg  
  inflating: brain_tumor_dataset/no/25 no.jpg  
  inflating: brain_tumor_dataset/no/26 no.jpg  
  inflating: brain_tumor_dataset/no/27 no.jpg  
  inflating: brain_tumor_datase

In [ ]:
from keras.layers import Input, Lambda,Conv2D,MaxPooling2D,Dropout,ZeroPadding2D,BatchNormalization,Flatten,Activation,Dense,MaxPooling2D
from keras.models import Model,Sequential
from keras.applications import InceptionV3,NASNetLarge,NASNetMobile,DenseNet121,Xception
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator,img_to_array,array_to_img
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [ ]:
!mkdir train_dataset
!mkdir test_dataset

os.mkdir('/content/train_dataset/yes')
os.mkdir('/content/train_dataset/no')

os.mkdir('/content/test_dataset/yes')
os.mkdir('/content/test_dataset/no')

In [ ]:
print(len(os.listdir('/content/no')))
print(len(os.listdir('/content/yes')))

98
155


In [ ]:
file_no=os.listdir('/content/no')
file_yes=os.listdir('/content/yes')

print(file_no)
print(file_yes)

['41 no.jpg', 'no 1.jpg', 'N20.JPG', '47 no.jpg', 'No17.jpg', '12 no.jpg', '29 no.jpg', '17 no.jpg', 'N21.jpg', '20 no.jpg', '38 no.jpg', '15 no.jpg', 'No16.jpg', '1 no.jpeg', 'N6.jpg', '45 no.jpg', 'no 2.jpg', '46 no.jpg', 'No20.jpg', '23 no.jpg', '19 no.jpg', '34 no.jpg', 'no.jpg', '48 no.jpeg', 'no 8.jpg', '24 no.jpg', 'no 9.png', 'N3.jpg', '9 no.jpg', 'N26.JPG', '21 no.jpg', 'no 923.jpg', 'N5.jpg', '36 no.jpg', '31 no.jpg', 'no 6.jpg', '37 no.jpg', 'no 98.jpg', '8 no.jpg', 'no 100.jpg', '7 no.jpg', 'No14.jpg', '33 no.jpg', 'No11.jpg', 'N22.JPG', 'No19.jpg', 'N1.JPG', 'no 94.jpg', '30 no.jpg', '10 no.jpg', 'N2.JPG', '5 no.jpg', '39 no.jpg', 'no 5.jpeg', '32 no.jpg', 'no 91.jpeg', '50 no.jpg', '42 no.jpg', '14 no.jpg', '27 no.jpg', '6 no.jpg', 'no 7.jpeg', 'No18.jpg', 'no 3.jpg', 'no 4.jpg', '35 no.jpg', 'N11.jpg', 'N19.JPG', 'No22.jpg', '26 no.jpg', 'no 10.jpg', 'no 99.jpg', '18 no.jpg', 'no 92.jpg', '11 no.jpg', 'N16.jpg', 'no 89.jpg', 'N17.jpg', '3 no.jpg', 'No12.jpg', 'No21.jpg',

In [ ]:
for file_name in file_yes[:110]:
  shutil.move(os.path.join('/content/yes', file_name), '/content/train_dataset/yes')

for file_name in file_yes[110:]:
  shutil.move(os.path.join('/content/yes', file_name), '/content/test_dataset/yes')

for file_name in file_no[:70]:
  shutil.move(os.path.join('/content/no', file_name), '/content/train_dataset/no')

for file_name in file_no[70:]:
  shutil.move(os.path.join('/content/no', file_name), '/content/test_dataset/no')

In [ ]:
def preprocess_input(x):
  img=keras.applications.xception.preprocess_input(img_to_array(x))
  return array_to_img(x)

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

training_set = train_datagen.flow_from_directory('/content/train_dataset',
                                                 target_size = (224, 224),
                                                 batch_size = 16,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('/content/test_dataset',
                                            target_size = (224, 224),
                                            batch_size = 16,
                                            class_mode = 'categorical')

Found 180 images belonging to 2 classes.
Found 73 images belonging to 2 classes.


In [ ]:
model_3=Xception(include_top=False,weights="imagenet",input_shape=(224, 224, 3),pooling='max',classes=1000)
folders = glob('/content/train_dataset/*')

x = Flatten()(model_3.output)
prediction = Dense(len(folders), activation='softmax')(x)

Yashvi = Model(inputs=model_3.input, outputs=prediction)
Yashvi.summary()

83689472/83683744 [==============================] - 2s 0us/step
Model: "functional_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 864         input_6[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 111, 111, 32) 0           block1_conv1_bn[0][0]            
______________________

In [ ]:
Yashvi.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy']) 
Yashvi.fit_generator(training_set,validation_data=test_set,epochs=10,steps_per_epoch=len(training_set),validation_steps=(len(test_set)))

Epoch 1/10
12/12 [==============================] - 4s 364ms/step - loss: 0.5516 - accuracy: 0.7444 - val_loss: 3.6723 - val_accuracy: 0.4384
Epoch 2/10
12/12 [==============================] - 4s 293ms/step - loss: 0.5018 - accuracy: 0.8667 - val_loss: 1.5735 - val_accuracy: 0.5616
Epoch 3/10
12/12 [==============================] - 3s 290ms/step - loss: 0.2759 - accuracy: 0.9389 - val_loss: 2.7898 - val_accuracy: 0.6986
Epoch 4/10
12/12 [==============================] - 4s 297ms/step - loss: 0.2438 - accuracy: 0.9278 - val_loss: 3.9471 - val_accuracy: 0.6164
Epoch 5/10
12/12 [==============================] - 3s 289ms/step - loss: 0.3752 - accuracy: 0.8722 - val_loss: 1.8063 - val_accuracy: 0.8630
Epoch 6/10
12/12 [==============================] - 4s 296ms/step - loss: 0.2107 - accuracy: 0.9222 - val_loss: 1.6419 - val_accuracy: 0.8493
Epoch 7/10
12/12 [==============================] - 4s 297ms/step - loss: 0.1284 - accuracy: 0.9333 - val_loss: 1.8942 - val_accuracy: 0.8767
Epoch 

In [ ]:
Yashvi.save('braintumuor.h5')

In [ ]:
def build_model(input_shape):
    X_input = Input(input_shape) 
    X = ZeroPadding2D((2, 2))(X_input) 
    
    X = Conv2D(32, (7, 7), strides = (1, 1))(X)
    X = BatchNormalization(axis = 3, name = 'bn0')(X)
    X = Activation('relu')(X) 
    
    X = MaxPooling2D((4, 4))(X) 
    X = MaxPooling2D((4, 4))(X) 
    X = Flatten()(X) 
    X = Dense(1, activation='sigmoid')(X) 
    model = Model(inputs = X_input, outputs = X)
    
    return model

In [ ]:
model = build_model((224,224,3))
print(model.summary())

Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
zero_padding2d_10 (ZeroPaddi (None, 228, 228, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 222, 222, 32)      4736      
_________________________________________________________________
bn0 (BatchNormalization)     (None, 222, 222, 32)      128       
_________________________________________________________________
activation_520 (Activation)  (None, 222, 222, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 55, 55, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)       

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy']) 
model.fit_generator(training_set,validation_data=test_set,epochs=10,steps_per_epoch=len(training_set),validation_steps=(len(test_set)))

Epoch 1/10
12/12 [==============================] - 3s 217ms/step - loss: 1.1921e-07 - accuracy: 0.5000 - val_loss: 1.1921e-07 - val_accuracy: 0.5000
Epoch 2/10
12/12 [==============================] - 2s 202ms/step - loss: 1.1921e-07 - accuracy: 0.5000 - val_loss: 1.1921e-07 - val_accuracy: 0.5000
Epoch 3/10
12/12 [==============================] - 2s 204ms/step - loss: 1.1921e-07 - accuracy: 0.5000 - val_loss: 1.1921e-07 - val_accuracy: 0.5000
Epoch 4/10
12/12 [==============================] - 2s 204ms/step - loss: 1.1921e-07 - accuracy: 0.5000 - val_loss: 1.1921e-07 - val_accuracy: 0.5000
Epoch 5/10
12/12 [==============================] - 2s 204ms/step - loss: 1.1921e-07 - accuracy: 0.5000 - val_loss: 1.1921e-07 - val_accuracy: 0.5000
Epoch 6/10
12/12 [==============================] - 2s 203ms/step - loss: 1.1921e-07 - accuracy: 0.5000 - val_loss: 1.1921e-07 - val_accuracy: 0.5000
Epoch 7/10
12/12 [==============================] - 2s 203ms/step - loss: 1.1921e-07 - accuracy: 0.5